# L4: Support Data Insight Analysis

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

## Initial Imports

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from utils import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Loading Tasks and Agents YAML files

In [15]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Using FileReadTool

In [16]:
from crewai_tools import FileReadTool
csv_tool = FileReadTool(file_path='./support_tickets_data.csv')

## Creating Agents, Tasks and Crew

In [17]:
# Creating Agents
suggestion_generation_agent = Agent(
  config=agents_config['suggestion_generation_agent'],
  tools=[csv_tool]
)

reporting_agent = Agent(
  config=agents_config['reporting_agent'],
  tools=[csv_tool]
)

chart_generation_agent = Agent(
  config=agents_config['chart_generation_agent'],
  allow_code_execution=True
)

# Creating Tasks
suggestion_generation = Task(
  config=tasks_config['suggestion_generation'],
  agent=suggestion_generation_agent
)

table_generation = Task(
  config=tasks_config['table_generation'],
  agent=reporting_agent
)

chart_generation = Task(
  config=tasks_config['chart_generation'],
  agent=chart_generation_agent
)

final_report_assembly = Task(
  config=tasks_config['final_report_assembly'],
  agent=reporting_agent,
  context=[suggestion_generation, table_generation, chart_generation]
)


# Creating Crew
support_report_crew = Crew(
  agents=[
    suggestion_generation_agent,
    reporting_agent,
    chart_generation_agent
  ],
  tasks=[
    suggestion_generation,
    table_generation,
    chart_generation,
    final_report_assembly
  ],
  verbose=True
)


2025-07-12 13:50:34,472 - 2872 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


## Testing our Crew

In [18]:
# support_report_crew.test(n_iterations=1, openai_model_name='gpt-4o')

## Training your crew and agents

In [19]:
# support_report_crew.train(n_iterations=1, filename='training.pkl')

## Comparing new test results

In [20]:
# support_report_crew.test(n_iterations=1, openai_model_name='gpt-4o')

In [21]:
# # Display the Trello screenshot
# from IPython.display import Image, display

# # Load and display the image
# test_image = Image(filename='test_before_training.png', width=368)
# display(test_image)

## Kicking off Crew

In [22]:
result = support_report_crew.kickoff()

# Agent: Suggestion Engine
## Task: Generate actionable suggestions for resolving each classified support ticket. The suggestions should be based on: - Issue Type: Tailor suggestions to the specific type of issue reported. - Historical Data: Use historical data such as resolution_time_minutes and
  satisfaction_rating to inform the suggestions.
- Customer Feedback: Incorporate insights from customer_comments to
  customize the suggestions further.

The goal is to provide clear, actionable steps that the support team can take to resolve each issue efficiently and effectively.



# Agent: Suggestion Engine
## Thought: I need to gather the support ticket data to analyze the issues and generate actionable suggestions based on the specified criteria.
## Using tool: Read a file's content
## Tool Input: 
"{\"name\": \"./support_tickets_data.csv\", \"args_schema\": {}}"
## Tool Output: 
ticket_id,customer_id,issue_type,issue_description,priority,date_submitted,response_time_minutes,resolution_

## Result

In [23]:
print(result)

### Comprehensive Support Report

#### Issue Classification Results
| Issue Type        | Frequency | Priority Level |
|-------------------|-----------|----------------|
| API Issue         | 10        | High           |
| Login Issue       | 9         | Critical       |
| Report Generation  | 7         | High           |
| Data Import       | 11        | High           |
| Feature Request   | 10        | High           |
| Billing Issue     | 10        | Critical       |
| UI Bug            | 7         | Medium         |

**Charts:**
1. **Issue Distribution**
   ![Issue Distribution](issue_distribution.png)

2. **Priority Levels**
   ![Priority Levels](priority_levels.png)

---

#### Agent Performance
| Agent ID | Resolved Tickets | Avg Resolution Time (minutes) | Avg Customer Satisfaction Score |
|----------|------------------|-------------------------------|---------------------------------|
| A001     | 7                | 826                           | 4.0                         

In [24]:
from IPython.display import display, Markdown

display(Markdown(result.raw))

### Comprehensive Support Report

#### Issue Classification Results
| Issue Type        | Frequency | Priority Level |
|-------------------|-----------|----------------|
| API Issue         | 10        | High           |
| Login Issue       | 9         | Critical       |
| Report Generation  | 7         | High           |
| Data Import       | 11        | High           |
| Feature Request   | 10        | High           |
| Billing Issue     | 10        | Critical       |
| UI Bug            | 7         | Medium         |

**Charts:**
1. **Issue Distribution**
   ![Issue Distribution](issue_distribution.png)

2. **Priority Levels**
   ![Priority Levels](priority_levels.png)

---

#### Agent Performance
| Agent ID | Resolved Tickets | Avg Resolution Time (minutes) | Avg Customer Satisfaction Score |
|----------|------------------|-------------------------------|---------------------------------|
| A001     | 7                | 826                           | 4.0                             |
| A002     | 7                | 551                           | 2.6                             |
| A003     | 10               | 575                           | 2.9                             |
| A004     | 14               | 513                           | 3.2                             |
| A005     | 10               | 1065                          | 3.5                             |

**Charts:**
3. **Average Resolution Time by Agent**
   ![Average Resolution Time](avg_resolution_time.png)

4. **Agent Performance: Resolution Time vs Customer Satisfaction**
   ![Agent Performance](agent_performance.png)

---

#### Customer Satisfaction
| Month      | Avg Satisfaction Rating |
|------------|-------------------------|
| January    | 3.3                     |
| February   | 2.7                     |
| March      | 3.6                     |
| April      | 3.6                     |
| May        | 2.8                     |
| June       | 3.9                     |
| July       | Data not available      |

**Chart:**
5. **Customer Satisfaction Ratings Over Time**
   ![Customer Satisfaction Ratings](customer_satisfaction.png)

---

#### Suggested Actions
- **API Issues**: Ensure swift follow-ups and monitor for recurring issues.
- **Login Issues**: Set up monitoring on the login system and ensure comprehensive repairs.
- **Report Generation**: Implement quicker escalation processes and regular updates to customers.
- **Data Import**: Address customer frustrations and document positive outcomes; ensure staff training for rapid resolutions.
- **Feature Requests**: Increase resource allocation and keep customers updated on progress.
- **Billing Issues**: Prioritize resolutions, communicate on ongoing issues, and create a taskforce for critical issues.
- **UI Bugs**: Investigate and create a remediation plan for identified bugs while implementing feedback loops.

---

This report integrates important performance metrics and actionable insights from the support data, formatted clearly for stakeholder consumption.

In [5]:
from IPython.display import display, Markdown

with open('final_report_3.md', 'r') as f:
    content = f.read()
display(Markdown(content))


**Final Report: Support System Insights**

---

### Key Findings

#### Order Status Distribution
| Status     | Count     |
|------------|-----------|
| Shipped    | 109       |
| Disputed   | 7         |
| Cancelled  | 3         |
| In Process | 2         |
| Resolved   | 2         |

*Chart: Order Status Distribution*  
![Order Status Distribution](charts/order_status_distribution.png)

#### Order Quantity by Product Line
| Product Line      | Total Orders | Total Quantity | Average Quantity per Order |
|------------------|--------------|----------------|----------------------------|
| Motorcycles      | 50           | 1231           | 24.62                      |
| Classic Cars     | 51           | 1242           | 24.35                      |
| Trucks and Buses | 51           | 1170           | 22.98                      |
| Vintage Cars     | 51           | 827            | 16.24                      |

#### Total Sales by Product Line
| Product Line      | Total Sales ($) |
|------------------|------------------|
| Motorcycles      | 162,970.42       |
| Classic Cars     | 156,734.90       |
| Trucks and Buses | 135,880.56       |
| Vintage Cars     | 71,812.86        |

*Chart: Total Sales by Product Line*  
![Total Sales by Product Line](charts/total_sales_by_product_line.png)

---

### Performance Insights

#### Average Sales per Order
| Order Status | Average Sales ($) |
|--------------|--------------------|
| Shipped      | 1,224.64           |
| Disputed     | 3,398.71           |
| Cancelled    | 1,600.76           |
| In Process   | 3,901.60           |
| Resolved     | 3,000.20           |

*Chart: Average Sales per Order by Order Status*  
![Average Sales per Order](charts/average_sales_per_order.png)

#### Cancellations by Product Line
| Product Line  | Total Cancellations |
|---------------|----------------------|
| Motorcycles   | 1                    |
| Classic Cars  | 2                    |
| Trucks and Buses  | 0                |
| Vintage Cars  | 1                    |

*Chart: Cancellations by Product Line*  
![Cancellations by Product Line](charts/cancellations_by_product_line.png)

---

### Satisfaction or Feedback Trends

#### Disputed Orders Over Time
| Year | Total Disputed Orders |
|------|-----------------------|
| 2003 | 3                     |
| 2004 | 4                     |
| 2005 | 0                     |

*Chart: Disputed Orders Over Time*  
![Disputed Orders Over Time](charts/disputed_orders_over_time.png)

---

### Suggested Actions
1. **High Volume of "Disputed" Orders**: Improve billing clarity and customer service response times.
2. **Inconsistent Shipping Times**: Analyze shipping logistics, and implement proactive tracking mechanisms.
3. **High Rate of Small Orders with Medium Margins**: Introduce volume discounts and promote product bundles.
4. **Cancellation of Orders**: Collect feedback on cancellations to identify areas for improvement in the purchasing experience.
5. **Underperformance in Specific Territories**: Tailor marketing strategies to meet local market needs and engage local partners.
6. **Repeated Returns of Products**: Investigate return reasons, enhance product descriptions, and improve customer education.
7. **Limited Engagement with Large Deal Sizes**: Assign dedicated account managers for targeted outreach to large clients.
8. **Low Sales in Certain Product Lines**: Conduct market research to phase out underperforming products and prioritize trending ones.
9. **Unexplained Variations in Sale Prices**: Implement consistent pricing strategies across platforms to stabilize customer perception.

---

This report synthesizes the performance of the support system, presenting data-driven insights to guide decision-making on future strategies for improvement and customer satisfaction.